# Tacotron: Towards End-to-End Speech Synthesis

Link: https://arxiv.org/abs/1703.10135

Authors: Yuxuan Wang, RJ Skerry-Ryan, Daisy Stanton, Yonghui Wu, Ron J. Weiss, Navdeep Jaitly, Zongheng Yang, Ying Xiao, Zhifeng Chen, Samy Bengio, Quoc Le, Yannis Agiomyrgiannakis, Rob Clark, Rif A. Saurous

Institution: Google, Inc.

Publication: arXiv

Date: 6 Apr 2017




## Background Materials

tacos & sushi


## What is this paper about?


Tacotron, an end-to-end generative text-to-speech model that synthesizes speech directly from characters.

## What is the motivation of this research?

Traditional TTS pipelines are complex. For example, it includes a text frontend extracting various linguistic features, a duration model, an acoustic feature prediction, and signal-processing based vocoder. They require domain expertise and labor.

## What makes this paper different from previous research?


Tacotron uses seq2seq with attention but does not require phoneme-level alignment, so there is no need to use pre-trained aligner.

Tacotron directly predict raw spectrogram so does not use vocoder.

Tacotron is complete end-to-end model, which can learn directly from `<text, audio>` pair to predict spectrogram, so can be trained completely from scratch.

Tacotron predicts frame level spectrograms, so it is significantly faster than sample level models but relatively low quality in terms of naturalness.

## How this paper achieve it?




## Dataset used in this study




## Implementations

- https://github.com/Kyubyong/tacotron


## Further Readings

- Sequence to sequence learning with neural networks https://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf
- Highway Networks https://arxiv.org/abs/1505.00387
- Fully Character-Level Neural Machine Translation without Explicit Segmentation https://arxiv.org/abs/1610.03017